In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
s = open('msr_train.txt').read()
# .decode('gbk')
s = s.split('\r\n')

def clean(s): #整理一下数据，有些不规范的地方
    if u'“/s'  in s:
        return s.replace(u' ”/s', '')
    elif u'”/s'  in s:
        return s.replace(u'“/s ', '')
    elif u'‘/s'  in s:
        return s.replace(u' ’/s', '')
    elif u'’/s'  in s:
        return s.replace(u'‘/s ', '')
    else:
        return s

s = u''.join(map(clean, s))
s = re.split(u'[，。！？、]/[bems]', s)

data = [] #生成训练样本
label = []
def get_xy(s):
    s = re.findall('(.)/(.)', s)
    if s:
        s = np.array(s)
        return list(s[:,0]), list(s[:,1])

for i in s:
    x = get_xy(i)
    if x:
        data.append(x[0])
        label.append(x[1])

In [3]:
maxlen = 32
d = pd.DataFrame(index=range(len(data)))
d['data'] = data
d['label'] = label
d = d[d['data'].apply(len)<=maxlen]
d.index = range(len(d))
tag = pd.Series({'s':0, 'b':1, 'm':2, 'e':3, 'x':4})

In [4]:
d.head()

,data,label
0,"[“, 人, 们, 常, 说, 生, 活, 是, 一, 部, 教, 科, 书]","[s, b, e, s, s, b, e, s, s, s, b, m, e]"
1,"[而, 血, 与, 火, 的, 战, 争, 更, 是, 不, 可, 多, 得, 的, 教, ...","[s, s, s, s, s, b, e, s, s, b, m, m, e, s, b, ..."
2,"[她, 确, 实, 是, 名, 副, 其, 实, 的, ‘, 我, 的, 大, 学, ’]","[s, b, e, s, b, m, m, e, s, s, s, s, b, e, s]"
3,"[“, 心, 静, 渐, 知, 春, 似, 海]","[s, s, s, s, s, s, s, s]"
4,"[花, 深, 每, 觉, 影, 生, 香]","[s, s, s, s, s, s, s]"


In [ ]:
chars = [] #统计所有字，跟每个字编号
for i in data:
    chars.append(i)
    

chars = pd.Series(chars).value_counts()
chars[:] = range(1,len(chars)+1)

#生成合适模型输入的格式
from keras.utils import np_utils
d['x'] = d['data'].apply(lambda x:np.array(list(chars[x])+[0]*(maxlen-len(x))))

def trans_one(x):
    _ = map(lambda x:np_utils.to_categorical(y,5),tag[x].reshape((-1,1)))
    _ = list(_)
    _.extend([np.array([[0,0,0,0,1]])]*(maxlen-len(x)))
    return np.array(_)

d['y'] = d['label'].apply(trans_one)


In [ ]:
d.head()

In [ ]:
#设计模型
word_size = 128
from keras.layers import Dense, Embedding, LSTM, TimeDistributed, Input, Bidirectional
from keras.models import Model



sequence = Input(shape=(maxlen,), dtype='int32')
embedded = Embedding(len(chars)+1, word_size, input_length=maxlen, mask_zero=True)(sequence)
blstm = Bidirectional(LSTM(64, return_sequences=True), merge_mode='sum')(embedded)
output = TimeDistributed(Dense(5, activation='softmax'))(blstm)
model = Model(input=sequence, output=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


batch_size = 1024
history = model.fit(np.array(list(d['x'])), np.array(list(d['y'])).reshape((-1,maxlen,5)), batch_size=batch_size, nb_epoch=50)


#转移概率，单纯用了等概率
zy = {'be':0.5, 
      'bm':0.5, 
      'eb':0.5, 
      'es':0.5, 
      'me':0.5, 
      'mm':0.5,
      'sb':0.5, 
      'ss':0.5
     }
zy = {i:np.log(zy[i]) for i in zy.keys()}

def viterbi(nodes):
    paths = {'b':nodes[0]['b'], 's':nodes[0]['s']}
    for l in range(1,len(nodes)):
        paths_ = paths.copy()
        paths = {}
        for i in nodes[l].keys():
            nows = {}
            for j in paths_.keys():
                if j[-1]+i in zy.keys():
                    nows[j+i]= paths_[j]+nodes[l][i]+zy[j[-1]+i]
            k = np.argmax(nows.values())
            paths[nows.keys()[k]] = nows.values()[k]
    return paths.keys()[np.argmax(paths.values())]

def simple_cut(s):
    if s:
        r = model.predict(np.array([list(chars[list(s)].fillna(0).astype(int))+[0]*(maxlen-len(s))]), verbose=False)[0][:len(s)]
        r = np.log(r)
        nodes = [dict(zip(['s','b','m','e'], i[:4])) for i in r]
        t = viterbi(nodes)
        words = []
        for i in range(len(s)):
            if t[i] in ['s', 'b']:
                words.append(s[i])
            else:
                words[-1] += s[i]
        return words
    else:
        return []

not_cuts = re.compile(u'([\da-zA-Z ]+)|[。，、？！\.\?,!]')
def cut_word(s):
    result = []
    j = 0
    for i in not_cuts.finditer(s):
        result.extend(simple_cut(s[j:i.start()]))
        result.append(s[i.start():i.end()])
        j = i.end()
    result.extend(simple_cut(s[j:]))
    return result
            
